mas temas de sql

In [2]:
import pandas as pd
import sqlite3 as sql 
from pym import tools as t

# 1. DISTINCT: se le aplica a una columna
# a. Valores unicos de esa columna
# b. Eliminar duplicados

# Selecciona TODAS las columnas de la tabla ventas
query = """
SELECT 
    --- Esto es un comentario en SQL
    * --- TODAS LAS COLUMNAS
FROM 
    ventas
"""
t.consultar_bbdd(query)

,Fecha,Producto,Clave,Cantidad,Precio,Total,Sucursal
0,2025-01-01,Cargadores,HUI-844,39,3954.10,154209.90,Milpa Alta
1,2025-01-01,Cuadernos,FOY-846,47,3444.24,161879.28,Santa Fé
2,2025-01-01,Papel,QTJ-739,10,506.73,5067.30,Centro
3,2025-01-01,Audífonos,UHM-612,26,6529.70,169772.20,Centro
4,2025-01-01,Borradores,EWV-583,31,7461.07,231293.17,Polanco
...,...,...,...,...,...,...,...
260249,2025-01-31,Libretas,BHL-911,30,454.89,13646.70,Milpa Alta
260250,2025-01-31,Baterías,WHS-186,6,3709.88,22259.28,Milpa Alta
260251,2025-01-31,Plumones,TKH-411,29,594.06,17227.74,Cuemanco
260252,2025-01-31,Mochilas,WEP-443,16,1128.27,18052.32,Coapa


In [3]:
# Order by solito, el ordenamiento es de manera ascendente
query = """
SELECT 
    DISTINCT Fecha
FROM 
    ventas
--- Sentencia de ordenamiento
ORDER BY
    Fecha DESC
--- limit n, te muestra las primeras n filas
LIMIT 5
"""
t.consultar_bbdd(query)

,Fecha
0,2025-01-31 00:00:00
1,2025-01-31
2,2025-01-30 00:00:00
3,2025-01-30
4,2025-01-29 00:00:00


In [4]:
# Las 5 primeras ventas del 6 de enero en Polanco
# WHERE ---> filtros

# Seleccioname todas las columnas de la tabla ventas
# donde la sucursal sea Polando Y el dia sea el 6 de enero
#                               &
#                              and
# o ---> | ----> or
query = """
SELECT 
    Producto, 
    Cantidad,
    Total AS Total_Ventas
FROM 
    ventas
WHERE
    Sucursal = 'Polanco'
AND 
    Fecha = '2025-01-01'
LIMIT 5
"""
t.consultar_bbdd(query)

,Producto,Cantidad,Total_Ventas
0,Borradores,31,231293.17
1,Tablets,33,39628.71
2,Lápices,32,71732.16
3,Pc,2,202.82
4,Pinturas,10,59046.20


Jerarquía de instrucciones en SQL

SELECT
FROM
WHERE
ORDER BY
LIMIT 5
Funciones de agregación:

SUM
MIN/MAX
AVG
COUNT

In [5]:
# CAST() ---> te convierte una columna al tipo de dato que le indiques
# CAST(NOM_COL AS TIPO_DATO_NUEVO)
query = """
SELECT 
    CAST(SUM(Total) AS INTEGER) AS Total_Ventas_Enero_Global
FROM 
    ventas
"""
t.consultar_bbdd(query)

,Total_Ventas_Enero_Global
0,16607173200


Definir una función en Python
Subir a SQL

In [6]:
def separador_miles(numero):
    """
    Da formato a un número insertando separadores de miles.

    Parámetros:
    ----------
    numero : int o float
        El número al que se le desea aplicar el formato con separadores de miles.

    Retorna:
    -------
    str
        El número formateado como cadena con comas separando los miles.
    """
    return "{:,}".format(numero)

sep_miles = lambda numero: "{:,}".format(numero)

print(separador_miles(1000))
print(sep_miles(1000))

1,000
1,000


In [7]:
# Conectarse a la base de datos
con = sql.connect("ventas.db")
cursor = con.cursor()

# a. nombre de la funcion en SQL
# b. cuantos parametros tiene tu funcion
# c. pones la funcion de Python
con.create_function("separador_miles", 1, separador_miles)

query = """
SELECT 
    separador_miles(
        CAST(
            SUM(Total) 
        AS 
            INTEGER
        )
    ) AS Total_Ventas_Enero_Global
FROM 
    ventas
"""

# fetchone() ---> nos muestra solo un valor
consulta_01 = cursor.execute(query).fetchone()

# Guardamos cambios
con.commit()
# Cerramos la conexion
con.close()

consulta_01

('16,607,173,200',)

No puedes usar funciones creadas por ti mism@ con pandas, entonces debemos ver una alternativa

In [8]:
# Conectarse a la base de datos
con = sql.connect("ventas.db")
cursor = con.cursor()

# a. nombre de la funcion en SQL
# b. cuantos parametros tiene tu funcion
# c. pones la funcion de Python
con.create_function("separador_miles", 1, separador_miles)

query = """
SELECT 
    Producto, 
    Cantidad,
    Total AS Total_Ventas
FROM 
    ventas
WHERE
    Sucursal = 'Polanco'
AND 
    Fecha = '2025-01-06'
"""

# fetchone() ---> nos muestra solo un valor
# fetchall() ---> todas las filas disponibles
# fetchmany(n) ---> traeme los primeros n registros
consulta_02 = cursor.execute(query).fetchmany(5)

# Guardamos cambios
con.commit()
# Cerramos la conexion
con.close()

consulta_02

[('Pinceles', 35, 68520.9),
 ('Pc', 23, 123331.75),
 ('Tijeras', 41, 354926.34),
 ('Pegamento', 10, 1998.3),
 ('Cables', 17, 18936.81)]


##Resumen de códigos
WHERE: Realizar filtros. Va combinada usualmente con operadores lógicos (AND, OR).
ORDER BY: Realizar el ordenamiento de la tabla con base en una columna. Por defecto se ordena de manera ascendente. DESC, ordenas de manera descendente.
DISTINCT: elimina duplicados, te arroja los valores únicos de una columna.
LIMIT N: te muestra los primeros N registros
OFFSET: salto de página
LIMIT-OFFSET: paginación

In [10]:

query = """
SELECT 
    *
FROM 
    ventas
LIMIT 15
"""
t.consultar_bbdd(query)

,Fecha,Producto,Clave,Cantidad,Precio,Total,Sucursal
0,2025-01-01,Cargadores,HUI-844,39,3954.10,154209.90,Milpa Alta
1,2025-01-01,Cuadernos,FOY-846,47,3444.24,161879.28,Santa Fé
2,2025-01-01,Papel,QTJ-739,10,506.73,5067.30,Centro
3,2025-01-01,Audífonos,UHM-612,26,6529.70,169772.20,Centro
4,2025-01-01,Borradores,EWV-583,31,7461.07,231293.17,Polanco
5,2025-01-01,Pegamento,ECV-177,12,6431.01,77172.12,CU
6,2025-01-01,Bolsas,YSA-138,42,782.29,32856.18,Santa Fé
7,2025-01-01,Monitores,SME-278,38,307.89,11699.82,Milpa Alta
8,2025-01-01,Pegamento,FEJ-643,11,1356.22,14918.42,Xochimilco
9,2025-01-01,Teclados,VRF-593,26,2134.93,55508.18,Xochimilco


In [9]:
query = """
SELECT 
    *
FROM 
    ventas
LIMIT 5
OFFSET 10
"""
t.consultar_bbdd(query)

,Fecha,Producto,Clave,Cantidad,Precio,Total,Sucursal
0,2025-01-01,Tablets,TXF-934,33,1200.87,39628.71,Polanco
1,2025-01-01,Pegamento,VIN-287,14,680.00,9520.00,Centro
2,2025-01-01,Sacapuntas,DMT-425,14,3434.53,48083.42,Narvarte
3,2025-01-01,Bolsas,NHZ-584,16,151.82,2429.12,Milpa Alta
4,2025-01-01,Cajas,WGC-148,22,2840.51,62491.22,Milpa Alta


In [10]:
import sqlite3 as sql
import pandas as pd

#Funcion para hacer consultas en SQL
def consulta(query):
  con = sql.connect('ventas.db')
  consulta = pd.read_sql(query, con)
  con.close()
  return consulta

In [23]:
a = """
SELECT
        *, SUM(Total) As Total_Ventas
FROM
      ventas
WHERE
     Total>1000
 GROUP BY 
      Fecha, Sucursal

 """
df= consulta(a)

In [ ]:
# Exploración de datos con pandas 

In [24]:
# Primeros datos
df.head()

,Fecha,Producto,Clave,Cantidad,Precio,Total,Sucursal,Total_Ventas
0,2025-01-01,Pegamento,ECV-177,12,6431.01,77172.12,CU,50716357.98
1,2025-01-01,Papel,QTJ-739,10,506.73,5067.30,Centro,54499279.35
2,2025-01-01,Bolsas,KKN-175,33,3004.14,99136.62,Coapa,58486069.29
3,2025-01-01,Pinceles,YAK-759,15,218.84,3282.60,Cuemanco,56696965.20
4,2025-01-01,Cargadores,HUI-844,39,3954.10,154209.90,Milpa Alta,50163704.19


In [25]:
# Ultimos
df.tail()

,Fecha,Producto,Clave,Cantidad,Precio,Total,Sucursal,Total_Ventas
615,2025-01-31 00:00:00,Cartulinas,GDP-346,23,798.96,18376.08,Narvarte,7379019.39
616,2025-01-31 00:00:00,Cuadernos,UAO-653,18,3916.57,70498.26,Polanco,7972602.12
617,2025-01-31 00:00:00,Bolsas,APP-582,27,143.95,3886.65,Santa Fé,5233789.81
618,2025-01-31 00:00:00,Cuadernos,ZFD-627,40,186.89,7475.60,Xochimilco,6275314.85
619,2025-01-31 00:00:00,Pegamento,XCV-783,33,3429.27,113165.91,Zócalo,7059090.51


In [46]:
# Estadísticas basicas sobre columnas
df.describe()

,Cantidad,Precio,Total,Total_Ventas
count,620.000000,620.000000,620.000000,6.200000e+02
mean,26.467742,2648.027355,69494.669435,2.677731e+07
std,14.326132,2182.738464,76355.892159,2.075470e+07
min,1.000000,25.680000,1010.420000,4.260809e+06
25%,14.000000,790.707500,12404.170000,6.433584e+06
50%,28.000000,2086.025000,41532.585000,2.045416e+07
75%,39.000000,3970.357500,101463.057500,4.748177e+07
max,50.000000,9335.430000,404348.560000,6.286441e+07


In [29]:
!pip3 install 'pandas-profiling[notebook,html]'
#import pandas_profiling as pp

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.2/687.2 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 18.9 MB/s eta 0:00:00ta 0:00:01
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27161 sha256=9001e23f512c929df51124ce814917353ed522880d6b1de901a4a02535849550
  Stored in directory: /home/codespace/.cache/pip/wheels/5f/d4/d7/4189b07b5902ee9f3ce0dbb14909fbe8037c39d6c63ffd49c9
Successfully built htmlmin
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
 

In [32]:
!pip install ydata-profiling

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 54.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 48.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.2/539.2 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 57.3 MB/s eta 0:00:00:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Attempting uninstall: multimethod
    Found existing installation: multimethod 2.0
    Uninstalling multimethod-2.0:
      Successfully uninstalled multimethod-2.0
  Attempting uninstall: visions
    Found existing installation: visions 0.7.4
    Uninstalling visions-0.7.4:
      Successfully uninstalled visions-0.7.4
  Attempting uninstall: matplotlib
    F

In [40]:
# la estructura de filas y columnas 
df.shape

(620, 8)

In [42]:
df.columns

Index(['Fecha', 'Producto', 'Clave', 'Cantidad', 'Precio', 'Total', 'Sucursal',
       'Total_Ventas'],
      dtype='object')

In [43]:
df.index

RangeIndex(start=0, stop=620, step=1)

In [47]:
df["Sucursal"].head(5)

0            CU
1        Centro
2         Coapa
3      Cuemanco
4    Milpa Alta
Name: Sucursal, dtype: object

In [48]:
df[["Sucursal","Producto","Total"]]

,Sucursal,Producto,Total
0,CU,Pegamento,77172.12
1,Centro,Papel,5067.30
2,Coapa,Bolsas,99136.62
3,Cuemanco,Pinceles,3282.60
4,Milpa Alta,Cargadores,154209.90
...,...,...,...
615,Narvarte,Cartulinas,18376.08
616,Polanco,Cuadernos,70498.26
617,Santa Fé,Bolsas,3886.65
618,Xochimilco,Cuadernos,7475.60


In [49]:
df[df["Sucursal"]=="Xochimilco"]

,Fecha,Producto,Clave,Cantidad,Precio,Total,Sucursal,Total_Ventas
8,2025-01-01,Pegamento,FEJ-643,11,1356.22,14918.42,Xochimilco,50246203.84
18,2025-01-01 00:00:00,Baterías,RKI-359,25,8800.09,220002.25,Xochimilco,6482220.87
28,2025-01-02,Uniformes,VPU-349,43,535.67,23033.81,Xochimilco,46994089.46
38,2025-01-02 00:00:00,Tablets,SLC-152,10,317.04,3170.40,Xochimilco,6532598.20
48,2025-01-03,Lápices,RQA-898,19,172.69,3281.11,Xochimilco,59330714.17
...,...,...,...,...,...,...,...,...
578,2025-01-29 00:00:00,Lápices,BEV-914,49,407.22,19953.78,Xochimilco,7207862.40
588,2025-01-30,Libretas,AEL-246,12,4386.59,52639.08,Xochimilco,47336474.06
598,2025-01-30 00:00:00,Cartulinas,MQU-212,4,1982.25,7929.00,Xochimilco,4633082.98
608,2025-01-31,Tijeras,RKZ-646,43,446.79,19211.97,Xochimilco,33993287.85


In [50]:
#Hacer filtros con pandas
#Info xochimilco
Filtro = df["Sucursal"] == "Xochimilco"
df[Filtro]

,Fecha,Producto,Clave,Cantidad,Precio,Total,Sucursal,Total_Ventas
8,2025-01-01,Pegamento,FEJ-643,11,1356.22,14918.42,Xochimilco,50246203.84
18,2025-01-01 00:00:00,Baterías,RKI-359,25,8800.09,220002.25,Xochimilco,6482220.87
28,2025-01-02,Uniformes,VPU-349,43,535.67,23033.81,Xochimilco,46994089.46
38,2025-01-02 00:00:00,Tablets,SLC-152,10,317.04,3170.40,Xochimilco,6532598.20
48,2025-01-03,Lápices,RQA-898,19,172.69,3281.11,Xochimilco,59330714.17
...,...,...,...,...,...,...,...,...
578,2025-01-29 00:00:00,Lápices,BEV-914,49,407.22,19953.78,Xochimilco,7207862.40
588,2025-01-30,Libretas,AEL-246,12,4386.59,52639.08,Xochimilco,47336474.06
598,2025-01-30 00:00:00,Cartulinas,MQU-212,4,1982.25,7929.00,Xochimilco,4633082.98
608,2025-01-31,Tijeras,RKZ-646,43,446.79,19211.97,Xochimilco,33993287.85


In [53]:
inicio = '2025-01-16'
fin = '2025-01-31'

df_quincena = df[(df['Fecha'] >= inicio) & (df['Fecha'] <= fin)]